In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
from math import sqrt
import tensorflow as tf

#Reading in the csv file from the Github repo
url = "https://raw.githubusercontent.com/Andrewli1212/project-4-Italy-Housing/main/Output/sales.csv"
df= pd.read_csv(url)
df.head()

C:\Users\eurib\AppData\Local\Temp\ipykernel_34040\3010066668.py:11: DtypeWarning: Columns (10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df= pd.read_csv(url)


,region,city,quarter,price,datetime,parking_spaces,bathrooms_per_room,bathrooms,bedrooms,top_floor,...,common_garden,private_garden,alarm_system,doorman,pool,villa,entire_property,apartment,attic,lofts
0,Sardegna,Valledoria,Via Alessandro Volta,78000.0,2022-12-17,0,0.333333,1.0,3.0,0,...,1,0,0,0,0,0,0,1,0,0
1,Sardegna,San Teodoro,NaN,460000.0,2023-03-31,1,0.666667,2.0,3.0,1,...,0,0,0,1,1,0,0,1,0,0
2,Sardegna,Cagliari,Fiera - Monte Mixi,387000.0,2023-04-18,0,0.500000,2.0,4.0,0,...,0,0,0,0,0,0,0,1,0,0
3,Sardegna,Cagliari,Piazza Giovanni XXIII,257000.0,2023-04-14,0,0.400000,2.0,5.0,0,...,0,0,0,0,0,0,0,1,0,0
4,Sardegna,Budoni,Località Baia Sant'Anna,370000.0,2023-03-31,1,0.666667,2.0,3.0,0,...,0,1,0,0,0,1,0,0,0,0


In [2]:
# Print all the column names in the DataFrame
print(df.columns)

Index(['region', 'city', 'quarter', 'price', 'datetime', 'parking_spaces',
       'bathrooms_per_room', 'bathrooms', 'bedrooms', 'top_floor', 'state',
       'energy_rating', 'sea_view', 'centralized_warming', 'surface',
       'furnished', 'balcony', 'TV_system', 'external_exposure', 'optic_fiber',
       'electric_gate', 'cellar', 'common_garden', 'private_garden',
       'alarm_system', 'doorman', 'pool', 'villa', 'entire_property',
       'apartment', 'attic', 'lofts'],
      dtype='object')


In [3]:
columns_to_drop = ['quarter','state','energy_rating','centralized_warming','alarm_system','entire_property',
                   'bathrooms_per_room','state','private_garden','top_floor','external_exposure']
df = df.drop(columns=columns_to_drop)
# Drop rows with empty values in the entire DataFrame
df.dropna(inplace=True)


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 22287 entries, 0 to 22290
Data columns (total 22 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   region          22287 non-null  object 
 1   city            22287 non-null  object 
 2   price           22287 non-null  float64
 3   datetime        22287 non-null  object 
 4   parking_spaces  22287 non-null  int64  
 5   bathrooms       22287 non-null  float64
 6   bedrooms        22287 non-null  float64
 7   sea_view        22287 non-null  float64
 8   surface         22287 non-null  float64
 9   furnished       22287 non-null  int64  
 10  balcony         22287 non-null  int64  
 11  TV_system       22287 non-null  int64  
 12  optic_fiber     22287 non-null  int64  
 13  electric_gate   22287 non-null  int64  
 14  cellar          22287 non-null  int64  
 15  common_garden   22287 non-null  int64  
 16  doorman         22287 non-null  int64  
 17  pool            22287 non-null  int6

In [5]:
df.head()

,region,city,price,datetime,parking_spaces,bathrooms,bedrooms,sea_view,surface,furnished,...,optic_fiber,electric_gate,cellar,common_garden,doorman,pool,villa,apartment,attic,lofts
0,Sardegna,Valledoria,78000.0,2022-12-17,0,1.0,3.0,0.0,67.0,0,...,0,1,0,1,0,0,0,1,0,0
1,Sardegna,San Teodoro,460000.0,2023-03-31,1,2.0,3.0,0.0,75.0,1,...,0,0,0,0,1,1,0,1,0,0
2,Sardegna,Cagliari,387000.0,2023-04-18,0,2.0,4.0,0.0,160.0,0,...,0,0,1,0,0,0,0,1,0,0
3,Sardegna,Cagliari,257000.0,2023-04-14,0,2.0,5.0,0.0,136.0,0,...,0,0,0,0,0,0,0,1,0,0
4,Sardegna,Budoni,370000.0,2023-03-31,1,2.0,3.0,0.0,105.0,0,...,0,0,0,0,0,0,1,0,0,0


In [6]:
#generate our categorical variable lists
df_cat= df.dtypes[df.dtypes == "object"].index.tolist()
#check the number of unique values in each column
df[df_cat].nunique()

region        20
city        1429
datetime     430
dtype: int64

In [7]:
# Create a OneHotEncoderInstance
enc= OneHotEncoder(sparse_output=False, drop="first")
#fir and transform the OneHotEncoder using the categorical variable list
encode_df= pd.DataFrame(enc.fit_transform(df[df_cat]))
# Add the encoded variable names to the Dataframe
encode_df.columns = enc.get_feature_names_out(df_cat)
encode_df.head()

,region_Basilicata,region_Calabria,region_Campania,region_Emilia-romagna,region_Friuli-venezia-giulia,region_Lazio,region_Liguria,region_Lombardia,region_Marche,region_Molise,...,datetime_2023-11-03,datetime_2023-11-04,datetime_2023-11-05,datetime_2023-11-06,datetime_2023-11-07,datetime_2023-12-01,datetime_2023-12-04,datetime_2023-12-05,datetime_2023-12-06,datetime_2023-12-07
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
#merge on-hot encoded features and drop the originals
df= df.merge(encode_df, left_index=True, right_index=True)
df=df.drop(columns=df_cat, axis=1)
df.head()

,price,parking_spaces,bathrooms,bedrooms,sea_view,surface,furnished,balcony,TV_system,optic_fiber,...,datetime_2023-11-03,datetime_2023-11-04,datetime_2023-11-05,datetime_2023-11-06,datetime_2023-11-07,datetime_2023-12-01,datetime_2023-12-04,datetime_2023-12-05,datetime_2023-12-06,datetime_2023-12-07
0,78000.0,0,1.0,3.0,0.0,67.0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,460000.0,1,2.0,3.0,0.0,75.0,1,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,387000.0,0,2.0,4.0,0.0,160.0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,257000.0,0,2.0,5.0,0.0,136.0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,370000.0,1,2.0,3.0,0.0,105.0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
X = df.drop(columns=["price"])
y = df["price"]

# Split the preprocessed data into a training and testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [10]:
# Create the StandardScaler instance then fit and scale the data
scaler= StandardScaler()
X_train_scaled= scaler.fit_transform(X_train)
X_test_scaled= scaler.transform(X_test)

In [11]:
# Define the model - deep neural net
number_input_features = X_train.shape[1]
hidden_nodes_layer1= 100
hidden_nodes_layer2=50
hidden_nodes_layer3=25

nn= tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim= number_input_features, activation="relu"))
# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))
# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))
# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="linear"))
# Check model structure
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               189500    
                                                                 
 dense_1 (Dense)             (None, 50)                5050      
                                                                 
 dense_2 (Dense)             (None, 25)                1275      
                                                                 
 dense_3 (Dense)             (None, 1)                 26        
                                                                 
Total params: 195851 (765.04 KB)
Trainable params: 195851 (765.04 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [12]:
# Compile the model
nn.compile(loss="mean_absolute_error", optimizer="adam")

In [13]:
# Train the model
fit_model= nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
523/523 [==============================] - 1s 1ms/step - loss: 271286.0625
Epoch 2/100
523/523 [==============================] - 1s 1ms/step - loss: 197839.1406
Epoch 3/100
523/523 [==============================] - 1s 1ms/step - loss: 183849.3281
Epoch 4/100
523/523 [==============================] - 1s 1ms/step - loss: 176931.4062
Epoch 5/100
523/523 [==============================] - 1s 1ms/step - loss: 172469.0781
Epoch 6/100
523/523 [==============================] - 1s 1ms/step - loss: 168970.3906
Epoch 7/100
523/523 [==============================] - 1s 1ms/step - loss: 165914.1406
Epoch 8/100
523/523 [==============================] - 1s 1ms/step - loss: 163064.4219
Epoch 9/100
523/523 [==============================] - 1s 1ms/step - loss: 160295.3125
Epoch 10/100
523/523 [==============================] - 1s 1ms/step - loss: 157396.5781
Epoch 11/100
523/523 [==============================] - 1s 1ms/step - loss: 154417.5625
Epoch 12/100
523/523 [===================

In [14]:
# Make predictions on the test data
y_pred = nn.predict(X_test_scaled)

# Calculate the Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error (MAE): {mae}")

175/175 [==============================] - 0s 753us/step
Mean Absolute Error (MAE): 155378.73856706676


In [15]:
# Save the trained model
nn.save("Neural_Network_Model/nn_model.keras")